## 准备数据

In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0

        # Flatten images
    x = x.reshape(-1, 28 * 28)  # Flatten training images
    x_test = x_test.reshape(-1, 28 * 28)  # Flatten test images
    
    return (x, y), (x_test, y_test)

In [20]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [22]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.truncated_normal([784, 128], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([128]))
        self.W2 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))


    def __call__(self, x):
        ####################
        '''实现模型函数体,返回未归一化的logits'''
        ####################
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits

        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [24]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [33]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 0.8683883 ; accuracy 0.79246664
epoch 1 : loss 0.86699915 ; accuracy 0.7927
epoch 2 : loss 0.86561614 ; accuracy 0.793
epoch 3 : loss 0.8642392 ; accuracy 0.79328334
epoch 4 : loss 0.86286837 ; accuracy 0.79345
epoch 5 : loss 0.86150336 ; accuracy 0.79371667
epoch 6 : loss 0.86014444 ; accuracy 0.79408336
epoch 7 : loss 0.8587913 ; accuracy 0.79435
epoch 8 : loss 0.857444 ; accuracy 0.7946
epoch 9 : loss 0.8561026 ; accuracy 0.7949167
epoch 10 : loss 0.8547671 ; accuracy 0.79518336
epoch 11 : loss 0.8534375 ; accuracy 0.7956667
epoch 12 : loss 0.8521136 ; accuracy 0.79585
epoch 13 : loss 0.85079545 ; accuracy 0.7959833
epoch 14 : loss 0.8494831 ; accuracy 0.7962
epoch 15 : loss 0.8481762 ; accuracy 0.7962833
epoch 16 : loss 0.84687513 ; accuracy 0.79655
epoch 17 : loss 0.8455797 ; accuracy 0.79681665
epoch 18 : loss 0.8442899 ; accuracy 0.79705
epoch 19 : loss 0.8430056 ; accuracy 0.79735
epoch 20 : loss 0.84172684 ; accuracy 0.7974667
epoch 21 : loss 0.8404535 ; accurac